# textCNN

In [ ]:
## 1.检验阶段

### 1.1引入模块库

In [ ]:
from MyModule import Net
import cupy as cp

### 1.2生成自定义测试数据集
格式为：3 batch h=9 w=10 channels=1

In [ ]:
def loadTestData():        
    x = cp.random.rand(90).reshape(1,9,10,1)
    # print(x)
    return x
# loadTestData()

### 1.3定义网络结构

In [ ]:
# 卷积层
conv1 = Net.Conv(kernal=(3,10,1,2),stride=1)
conv2 = Net.Conv(kernal=(4,10,1,2),stride=1)
conv3 = Net.Conv(kernal=(5,10,1,2),stride=1)

# 激活函数Relu
relu1 = Net.Relu()
relu2 = Net.Relu()
relu3 = Net.Relu()

# 池化层
max_pooling1 = Net.Pool(ksize=2)
max_pooling2 = Net.Pool(ksize=2)
max_pooling3 = Net.Pool(ksize=2)

# Flatten 层
flatten1 = Net.Flatten()
flatten2 = Net.Flatten()
flatten3 = Net.Flatten()

# Concat融合
concat1 = Net.Concat()

# 全连接层
fc1 = Net.Fc(16,2)

# softmax
softmax = Net.Softmax()

In [ ]:
def Forward(X,Y):
    # 卷积
    forward1 = conv1.forward(X)
    forward2 = conv2.forward(X)
    forward3 = conv3.forward(X)
    # print(forward1.shape)
    # relu
    forward1 = relu1.forward(forward1)
    forward2 = relu2.forward(forward2)
    forward3 = relu3.forward(forward3)
    # print(forward1.shape)
    # 池化
    forward1 = max_pooling1.forward(forward1)
    forward2 = max_pooling2.forward(forward2)
    forward3 = max_pooling3.forward(forward3)
    # print(forward1.shape)
    # flatten
    forward1 = flatten1.forward(forward1)
    forward2 = flatten2.forward(forward2)
    forward3 = flatten3.forward(forward3)
    print(forward1.shape)
    # print(forward2.shape)
    # print(forward3.shape)
    # concate
    forward = concat1.forward(forward1,forward2,forward3)
    print(forward.shape)
    # 全连接
    forward = fc1.forward(forward)
    # print(forward.shape)
    # softmax
    loss,delta = softmax.calcuteLoss(forward,Y)
    predict = softmax.predict(forward)
    right = 0
    # print(predict.shape)
    for i in range(3):
        if cp.argmax(predict[i])==Y[i] :
            right = right + 1
    return loss,delta,right/3
# print(f"softmax_layer:{delta.shape}" )
# print(f"loss:{loss} delta:{delta}")

learning_rate = 0.1

In [ ]:
def Backward(delta):
    # 全连接
    backward = fc1.backward(delta,learning_rate)
    # print(f"fc:{backward.shape}")
    # concate
    backward1,backward2,backward3 =  concat1.backward(backward)
    # print(f"concate:{backward1.shape} {backward2.shape} {backward3.shape}")
    # flatten
    backward1 = flatten1.backward(backward1)
    backward2 = flatten2.backward(backward2)
    backward3 = flatten3.backward(backward3)
    # print(f"flatten1:{backward1.shape} flatten2:{backward2.shape} flatten3:{backward3.shape}")
    # 池化
    backward1 = max_pooling1.backward(backward1)
    backward2 = max_pooling2.backward(backward2)
    backward3 = max_pooling3.backward(backward3)
    # print(f"pool1:{backward1.shape} pool2:{backward2.shape} pool3:{backward3.shape}")
    # relu
    backward1 = relu1.backward(backward1)
    backward2 = relu2.backward(backward2)
    backward3 = relu3.backward(backward3)
    # 卷积
    backward1 = conv1.backward(backward1,learning_rate)
    backward2 = conv2.backward(backward2,learning_rate)
    backward3 = conv3.backward(backward3,learning_rate)
    # print(f"反向传播完成")
    

#### 1.4.3训练结果和损失展示

In [ ]:
for i in range(1001):
    data = loadTestData()
   
    loss,delta,right_rate= Forward(X,Y)
    Backward(delta)
    if(i%100==0) : print(f"epoch:{i}/{100} loss:{loss} right_rate :{right_rate} softmax:{softmax.softmax}")